Hello World!
This notebook describes the effort to cleanup and cluster a Habits dataset.

de-duplicate : 1. user i'd
               2. email i'd (prefer most recent, according to Age-on-platform
               3.


Clustering the Data:
two methods - 

A) cluster the data  and see how many clusters are there


B) three bins : "signed up not used", "completed not beyond 1 week", "everyone else"

# Looking around the data set

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# Import the required modules
import pandas as pd
import numpy as np
import scipy as sp

In [3]:
# simple function to read in the user data file.
# the argument parse_dates takes in a list of colums, which are to be parsed as date format
user_data_raw_csv = pd.read_csv("/home/eyebell/local_bin/janacare/datasets/Habits-Data_upto-7th-May.csv",\
                            parse_dates = [-3, -2, -1])

In [4]:
# import the pyexcel module
#import pyexcel as pe
#from pyexcel.ext import xls

# load the file
#records = pe.get_records(file_name="/home/eyebell/local_bin/janacare/datasets/Habits-Data_upto-7th-May.xls")
#len(records)
#for record in records:
    #print record

In [5]:
# data metrics
user_data_raw_csv.shape # Rows , colums


(53239, 9)

In [6]:
# data metrics
user_data_raw_csv.dtypes # data type of colums

user_id                   int64
first_name               object
last_name                object
username                 object
email                    object
phone_number             object
date_joined      datetime64[ns]
first_login      datetime64[ns]
last_activity    datetime64[ns]
dtype: object

In [7]:
user_data_to_clean = user_data_raw_csv

In [ ]:
# Some basic statistical information on the data
#user_data_to_clean.describe()

# Data Clean up

In the last section of looking around, I saw that a lot of rows do not have any values or have garbage values(see first row of the table above).
This can cause errors when computing anything using the values in these rows, hence a clean up is required.

If a the coulums *last_activity* and *first_login* are empty then drop the corresponding row !

In [8]:
# Lets check the health of the data set
user_data_to_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53239 entries, 0 to 53238
Data columns (total 9 columns):
user_id          53239 non-null int64
first_name       53108 non-null object
last_name        5360 non-null object
username         53239 non-null object
email            53089 non-null object
phone_number     53094 non-null object
date_joined      53239 non-null datetime64[ns]
first_login      50773 non-null datetime64[ns]
last_activity    44903 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(5)
memory usage: 3.7+ MB


As is visible from the last column (*age_on_platform*) data type, Pandas is not recognising it as date type format. 
This will make things difficult, so I delete this particular column and add a new one.
Since the data in *age_on_platform* can be recreated by doing *age_on_platform* = *last_activity* - *first_login* 

But on eyeballing I noticed some, cells of column *first_login* have greater value than corresponding cell of *last_activity*. These cells need to be swapped, since its not possible to have *first_login* > *last_activity*
Finally the columns *first_login*, *last_activity* have missing values, as evident from above table. Since this is time data, that in my opinion should not be imputed, we will drop/delete the columns.

In [9]:
# Run a loop through the data frame and check each row for this anamoly, if found drop,
# this is being done ONLY for selected columns

import datetime

swapped_count = 0
first_login_count = 0
last_activity_count = 0
email_count = 0
userid_count = 0

for index, row in user_data_to_clean.iterrows():        
        if row.last_activity == pd.NaT or row.last_activity != row.last_activity:
            last_activity_count = last_activity_count + 1
            #print row.last_activity
            user_data_to_clean.drop(index, inplace=True)

        elif row.first_login > row.last_activity:
            user_data_to_clean.drop(index, inplace=True)
            swapped_count = swapped_count + 1

        elif row.first_login != row.first_login or row.first_login == pd.NaT:
            user_data_to_clean.drop(index, inplace=True)
            first_login_count = first_login_count + 1

        elif row.email != row.email: #or row.email == '' or row.email == ' ':
            user_data_to_clean.drop(index, inplace=True)
            email_count = email_count + 1

        elif row.user_id != row.user_id:
            user_data_to_clean.drop(index, inplace=True)
            userid_count = userid_count + 1
    
#temp_date_var = row.first_login
#user_data_to_clean.set_value(index, 'first_login', row.last_activity)
#user_data_to_clean.set_value(index, 'last_activity', temp_date_var)
#print "to_swap: \t" + "first_login\t" + row.first_login.isoformat() + "\tlast_activity\t" + row.last_activity.isoformat()

print "last_activity_count=%d\tswapped_count=%d\tfirst_login_count=%d\temail_count=%d\tuserid_count=%d" \
% (last_activity_count, swapped_count, first_login_count, email_count, userid_count)

last_activity_count=8336	swapped_count=922	first_login_count=170	email_count=2	userid_count=0


In [10]:
user_data_to_clean.shape

(43809, 9)

In [11]:
# Create new column 'age_on_platform' which has the corresponding value in date type format
user_data_to_clean["age_on_platform"] = user_data_to_clean["last_activity"] - user_data_to_clean["first_login"]

In [12]:
user_data_to_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43809 entries, 0 to 53238
Data columns (total 10 columns):
user_id            43809 non-null int64
first_name         43806 non-null object
last_name          2385 non-null object
username           43809 non-null object
email              43809 non-null object
phone_number       43801 non-null object
date_joined        43809 non-null datetime64[ns]
first_login        43809 non-null datetime64[ns]
last_activity      43809 non-null datetime64[ns]
age_on_platform    43809 non-null timedelta64[ns]
dtypes: datetime64[ns](3), int64(1), object(5), timedelta64[ns](1)
memory usage: 3.7+ MB


#### Validate if email i'd is correctly formatted

In [14]:
from validate_email import validate_email

email_count_invalid = 0
email_count_duplicate = 0
for index, row in user_data_to_clean.iterrows():        
        if not validate_email(row.email):
            #print row.email
            user_data_to_clean.drop(index, inplace=True)
            email_count_invalid = email_count_invalid + 1
        
        #else:
         #   flag = 0
          #  for index_j, row_j in user_data_to_clean.iterrows():
           #     if row.email == row_j.email:
            #        flag = flag + 1
             #       if flag > 1:
              #          user_data_to_clean.drop(index_j, inplace=True)
               #         email_count_duplicate = email_count_duplicate + 1
print "Number of email-id invalid: %d & number od email-id duplicated: %d" % (email_count_invalid, email_count_duplicate)


Number of email-id invalid: 45 & number od email-id duplicated: 0


In [15]:
# Check the result of last operation 
user_data_to_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43612 entries, 0 to 53238
Data columns (total 10 columns):
user_id            43612 non-null int64
first_name         43609 non-null object
last_name          2300 non-null object
username           43612 non-null object
email              43612 non-null object
phone_number       43604 non-null object
date_joined        43612 non-null datetime64[ns]
first_login        43612 non-null datetime64[ns]
last_activity      43612 non-null datetime64[ns]
age_on_platform    43612 non-null timedelta64[ns]
dtypes: datetime64[ns](3), int64(1), object(5), timedelta64[ns](1)
memory usage: 3.7+ MB


### Remove duplicates

In [16]:
user_data_to_deDuplicate = user_data_to_clean

In [17]:
bool_list = user_data_to_deDuplicate.email.str.strip().duplicated()

In [35]:
user_data_deDuplicateD = user_data_to_deDuplicate.loc[~user_data_to_deDuplicate.email.str.strip().duplicated()]
len(user_data_deDuplicateD)

40495

In [36]:
user_data_deDuplicateD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40495 entries, 0 to 53238
Data columns (total 10 columns):
user_id            40495 non-null int64
first_name         40492 non-null object
last_name          2111 non-null object
username           40495 non-null object
email              40495 non-null object
phone_number       40487 non-null object
date_joined        40495 non-null datetime64[ns]
first_login        40495 non-null datetime64[ns]
last_activity      40495 non-null datetime64[ns]
age_on_platform    40495 non-null timedelta64[ns]
dtypes: datetime64[ns](3), int64(1), object(5), timedelta64[ns](1)
memory usage: 3.4+ MB


In [ ]:
# Save the labeled data frame as excel file
#from pandas import options
#options.io.excel.xlsx.writer = 'xlsxwriter'
#user_data_imputed_data_frame_labeled.to_excel('user_data_imputed_data_frame_labeled.xlsx')

## Clustering

In [ ]:
#user_data_imputed_labeled_numpy_matrix = user_data_imputed_data_frame_labeled.as_matrix()

In [31]:
from sklearn.cluster import MeanShift, estimate_bandwidth

#x = [1,1,5,6,1,5,10,22,23,23,50,51,51,52,100,112,130,500,512,600,12000,12230]
x = pd.Series(user_data_deDuplicateD[])

X = np.array(zip(x,np.zeros(len(x))), dtype=np.int)
bandwidth = estimate_bandwidth(X, quantile=0.1)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

for k in range(n_clusters_):
    my_members = labels == k
    print "cluster {0}: {1}".format(k, X[my_members, 0])

cluster 0: [ 1  1  5  6  1  5 10 22 23 23]
cluster 1: [50 51 51 52]
cluster 2: [100 112 130]
cluster 3: [500 512]
cluster 4: [12000]
cluster 5: [12230]
cluster 6: [600]


In [33]:
X = np.array(zip(x,np.zeros(len(x))), dtype=np.int)

In [34]:
print X

[[    1     0]
 [    1     0]
 [    5     0]
 [    6     0]
 [    1     0]
 [    5     0]
 [   10     0]
 [   22     0]
 [   23     0]
 [   23     0]
 [   50     0]
 [   51     0]
 [   51     0]
 [   52     0]
 [  100     0]
 [  112     0]
 [  130     0]
 [  500     0]
 [  512     0]
 [  600     0]
 [12000     0]
 [12230     0]]
